In [44]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import dataLoader as dl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import util as ut

#from Inner_Speech_Dataset.Plotting.ERPs import 
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Select_time_window, Transform_for_classificator, Split_trial_in_time
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Calculate_power_windowed
from Inner_Speech_Dataset.Python_Processing.Utilitys import picks_from_channels
from Inner_Speech_Dataset.Python_Processing.Data_processing import Average_in_frec

In [55]:


sampling_rate = 256
nr_of_datasets=1
data, labels = dl.load_multiple_datasets(nr_of_datasets=nr_of_datasets, sampling_rate=sampling_rate, t_min=2, t_max=3, twoDLabels=False)

ch_names = ut.get_channelNames()

print(data.shape)
data_p =  ut.get_power_array(data[:,:128,:], sampling_rate, trialSplit=1).squeeze()
print(data_p.shape)
#print(data_p[:,:,1])

#Getting Freq Data 
nr_of_buckets = 10
buckets = ut.createFreqBuckets(data[:,:128,:], nr_of_buckets)
print("buckets")
print(buckets)
data_f = ut.data_into_freq_buckets(data[:,:128,:], nr_of_buckets, buckets)
#data_f = np.concatenate([dataf, np.zeros([data_f.shape[0], nr_of_datasets, data_f.shape[2]])], axis=1)

print(data_f.shape)


## Normalize data



Data shape: [trials x channels x samples]
(500, 128, 256)
Labels shape
(500, 4)
Final data shape
(100, 128, 256)
Final labels shape
(100,)
Up is 0.0 and Down is 1.0
(100, 128, 256)
(100, 128, 2)
buckets
[[  0   2]
 [  2   7]
 [  7  13]
 [ 13  19]
 [ 19  28]
 [ 28  40]
 [ 40  59]
 [ 59  83]
 [ 83 113]
 [ 80  94]]
(100, 128, 10)


In [56]:
print(data_f.shape)#ss
print(data_p.shape)
print(labels.shape)
print(data_p[2,5])
data = np.concatenate([data_f, data_p], axis =2 ) #, data
print(data.shape)
data = keras.utils.normalize(data, axis=1, order=2)
print(data.shape)
#print(data[2,5])
#print(labels[2])
print(labels[:].shape)

(100, 128, 10)
(100, 128, 2)
(100,)
[9.56732240e-11 4.90024352e-11]
(100, 128, 12)
(100, 128, 12)
(100,)


In [47]:
#Add end of ekg with specific array for each person. 


In [57]:
#Splitting into training and test data
#print(labels)


order = np.arange(labels.shape[0])
np.random.shuffle(order)

temp_data = np.zeros(data.shape)
temp_labels = np.zeros(labels.shape)

for x in range(labels.shape[0]):
    i = order[x]
    
    temp_data[x] = data[i]
    temp_labels[x] = labels[i]

data = temp_data
labels = temp_labels

data_train, data_test = np.split(data, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
labels_train, labels_test = np.split(labels, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
print(labels_train.shape)
print(data_train.shape)
print(data_test.shape)




(75,)
(75, 128, 12)
(25, 128, 12)


In [65]:
#!pip3 install sklearn -q
from tabnanny import verbose
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report

def svmPipeline(data_train, data_test, labels_train,
    labels_test, kernel="linear", degree=3, gamma="auto", C = 1
    , coefs = np.zeros([1, data_train.shape[1]*data_train.shape[2]])):
    
    anova_filter = SelectKBest(f_classif, k=20)
    clf = make_pipeline(anova_filter, StandardScaler() ,  SVC(
        gamma=gamma, kernel=kernel, degree=degree, verbose=False, C=C))

    clf.fit(np.reshape(data_train, [data_train.shape[0], -1]), labels_train)
    predictions = clf.predict(np.reshape(data_test, [data_test.shape[0], -1]))
    
    print(classification_report(labels_test, predictions))
    if kernel == "linear":
        coefs = coefs + clf[:-1].inverse_transform(clf[-1].coef_)
    #print(clf[:-1].inverse_transform(clf[-1].coef_).shape)

    correct = np.zeros(labels_test.shape)
    correctamount=0
    for nr, pred in enumerate(predictions,0):
        if pred == labels_test[nr]:
            correct[nr] = 1
            correctamount +=1


    return correctamount/labels_test.shape[0], coefs

#[LibSVM]Result for gamma auto, kernel rbf, C = 0.8: 0.6
#[LibSVM]Result for gamma scale, kernel rbf, C = 0.8: 0.6
coefs = np.zeros([1, data_train.shape[1]*data_train.shape[2]])
for kernel in ["linear", "rbf", "sigmoid"]:

    for C in np.linspace(0.1,2,20):
    
        if kernel == "linear":
            for degree in range(1,5):
                res, coefs = svmPipeline(data_train, data_test, labels_train, 
                labels_test, degree=degree, kernel = kernel, C = C, coefs=coefs)
                print("Result for degree {}, kernel {}, C = {}: {}".format(degree, kernel, C,  res))
        # else:
        #     for gamma in ["auto", "scale"]:

        #         res = svmPipeline(data_train, data_test, labels_train, 
        #         labels_test, degree=degree, kernel = kernel, gamma=gamma, C = C)
        #         print("Result for gamma {}, kernel {}, C = {}: {}".format(gamma, kernel, C, res))

coefs = np.reshape(coefs, [128,-1])

              precision    recall  f1-score   support

         0.0       0.56      0.45      0.50        11
         1.0       0.62      0.71      0.67        14

    accuracy                           0.60        25
   macro avg       0.59      0.58      0.58        25
weighted avg       0.59      0.60      0.59        25

Result for degree 1, kernel linear, C = 0.1: 0.6
              precision    recall  f1-score   support

         0.0       0.56      0.45      0.50        11
         1.0       0.62      0.71      0.67        14

    accuracy                           0.60        25
   macro avg       0.59      0.58      0.58        25
weighted avg       0.59      0.60      0.59        25

Result for degree 2, kernel linear, C = 0.1: 0.6
              precision    recall  f1-score   support

         0.0       0.56      0.45      0.50        11
         1.0       0.62      0.71      0.67        14

    accuracy                           0.60        25
   macro avg       0.59      0

In [67]:

print(np.where(coefs!=0))
print(coefs.shape)

(array([  0,   9,  38,  39,  40,  41,  50,  51,  52,  62,  63,  78,  79,
       101, 103, 104, 105, 117, 117, 118], dtype=int64), array([ 3,  6,  3,  3,  3,  3,  3,  4,  4,  4,  4, 11, 10,  4,  5,  5,  5,
        1, 11,  8], dtype=int64))
(128, 12)


In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()





eeg_model = tf.keras.Sequential([

    layers.LocallyConnected1D(20, input_shape = (data_train.shape[1],data_train.shape[2]), 
     kernel_size=15, 
     padding="valid", strides=8,
     activation="relu"),
    
    
    layers.LocallyConnected1D(20, 
    kernel_size=15, 
    padding="valid", strides=5,
    activation="relu"),
    
    layers.Dropout(0.8),
    
    layers.Dense(units=200, activation="relu"),
    
    layers.Dropout(0.8),

    layers.Flatten(),

    layers.Dense(units=2, activation="softmax")


])
eeg_model.build()
eeg_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 locally_connected1d (Locall  (None, 15, 20)           54300     
 yConnected1D)                                                   
                                                                 
 locally_connected1d_1 (Loca  (None, 1, 20)            6020      
 llyConnected1D)                                                 
                                                                 
 dropout (Dropout)           (None, 1, 20)             0         
                                                                 
 dense (Dense)               (None, 1, 200)            4200      
                                                                 
 dropout_1 (Dropout)         (None, 1, 200)            0         
                                                                 
 flatten (Flatten)           (None, 200)               0

In [52]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
#callback2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=2)

eeg_model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])
#data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])

data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])
#data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])

outputs = eeg_model.fit(data_train, labels_train, validation_split=0.2 ,callbacks=[callback], epochs=50) #

print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

Epoch 1/50


ValueError: in user code:

    File "c:\Users\Luna\anaconda3\envs\ForMasterBelgium\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Luna\anaconda3\envs\ForMasterBelgium\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Luna\anaconda3\envs\ForMasterBelgium\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Luna\anaconda3\envs\ForMasterBelgium\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Luna\anaconda3\envs\ForMasterBelgium\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\Luna\anaconda3\envs\ForMasterBelgium\lib\site-packag

In [ ]:

eeg_model.save('saved_model/my_model6')

INFO:tensorflow:Assets written to: saved_model/my_model6\assets


In [ ]:
print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

result2 = []
print(result)
for res in result:
    x = np.maximum(res[0], res[1])
    result2.append(int(np.where(res == x)[0]))

result2 = np.array(result2)
print(result2)

Results
1/1 [==============================] - 0s 19ms/step
[[7.2694892e-01 2.7305105e-01]
 [3.8849583e-01 6.1150414e-01]
 [2.3448229e-02 9.7655177e-01]
 [1.0121168e-02 9.8987877e-01]
 [6.9411544e-05 9.9993062e-01]
 [1.4454442e-01 8.5545564e-01]
 [1.3466945e-02 9.8653305e-01]
 [7.4755740e-01 2.5244254e-01]
 [7.5335383e-02 9.2466462e-01]
 [8.3851330e-02 9.1614866e-01]
 [7.8725910e-01 2.1274088e-01]
 [2.0782053e-01 7.9217941e-01]
 [1.4555229e-02 9.8544472e-01]
 [5.3242538e-02 9.4675744e-01]
 [6.2527745e-03 9.9374723e-01]
 [5.6260753e-01 4.3739250e-01]
 [7.4348336e-01 2.5651667e-01]
 [7.6414049e-01 2.3585953e-01]
 [4.1789272e-01 5.8210737e-01]
 [3.9263193e-02 9.6073681e-01]
 [6.0369277e-01 3.9630726e-01]
 [7.6168078e-01 2.3831928e-01]
 [1.8715034e-01 8.1284964e-01]
 [4.8335150e-02 9.5166487e-01]
 [1.1280457e-01 8.8719541e-01]]
[0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 0 1 1 0 0 1 1 1]
